In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pip install pandas==1.3.1

In [7]:
!pip install neattext

In [8]:
!pip3 install  pyLDAvis --quiet

In [9]:
import pandas as pd
import numpy as np
from pprint import pprint

import matplotlib.pyplot as plt

# spacy for lemmatization
import spacy

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


import plotly.express as px
import nltk
nltk.download('all')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import neattext.functions as nf
from textblob import TextBlob, Word
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
from tqdm import tqdm
from collections import Counter
import ast

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sb

from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import scipy.stats as stats

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
output_notebook()

%matplotlib inline

from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
output_notebook()

%matplotlib inline
import re
import spacy
import string

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis # don't skip this

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working



In [10]:
path = '/content/drive/MyDrive/T5camp/Anime_project/'

anime = pd.read_csv(path + 'animes.csv')
# profiles = pd.read_csv(path + 'profiles.csv')
# reviews = pd.read_csv(path + 'reviews.csv')

In [11]:
pd.options.display.max_colwidth = 500
syn = anime['synopsis']
syn

0        Following their participation at the Inter-High, the Karasuno High School volleyball team attempts to refocus their efforts, aiming to conquer the Spring tournament instead.  \r\n \r\nWhen they receive an invitation from long-standing rival Nekoma High, Karasuno agrees to take part in a large training camp alongside many notable volleyball teams in Tokyo and even some national level players. By playing with some of the toughest teams in Japan, they hope not only to sharpen their skills, but ...
1        Music accompanies the path of the human metronome, the prodigious pianist Kousei Arima. But after the passing of his mother, Saki Arima, Kousei falls into a downward spiral, rendering him unable to hear the sound of his own piano. \r\n  \r\nTwo years later, Kousei still avoids the piano, leaving behind his admirers and rivals, and lives a colorless life alongside his friends Tsubaki Sawabe and Ryouta Watari. However, everything changes when he meets a beautiful violinist, Kaori

In [12]:
syn = syn.astype('str')
syn = syn.apply(lambda text: text.lower())
syn = syn.apply(nf.remove_multiple_spaces)
syn = syn.apply(nf.remove_puncts)
syn = syn.apply(nf.remove_stopwords)
syn = syn.apply(nf.remove_numbers)
syn = syn.apply(nf.remove_special_characters)

In [13]:
# make counter for words
from collections import Counter
cnt = Counter()
for text in syn.values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('source', 7131),
 ('world', 3530),
 ('school', 3381),
 ('new', 3153),
 ('mal', 2656),
 ('life', 2648),
 ('ann', 2530),
 ('written', 2503),
 ('rewrite', 2414),
 ('girl', 2395)]

In [14]:
#remove rare words
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])
syn = syn.apply(lambda text: remove_rarewords(text))
syn

0        following participation interhigh karasuno high school volleyball team attempts refocus efforts aiming conquer spring tournament instead receive invitation longstanding rival nekoma high karasuno agrees large training camp alongside notable volleyball teams tokyo national level players playing toughest teams japan hope sharpen skills come new attacks strengthen hinata kageyama attempt devise powerful weapon possibly break sturdiest blocks facing chance victory senior players graduate members...
1        music accompanies path human metronome prodigious pianist kousei arima passing mother saki arima kousei falls downward spiral rendering unable hear sound piano years later kousei avoids piano leaving admirers rivals lives colorless life alongside friends tsubaki sawabe ryouta watari changes meets beautiful violinist kaori miyazono stirs world sets journey face music based manga series shigatsu wa kimi uso approaches story kouseis recovery discovers music playing note perfectly 

In [15]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word,'v') for word in text.split()])

syn = syn.apply(lambda text: lemmatize_words(text))
syn

0        follow participation interhigh karasuno high school volleyball team attempt refocus efforts aim conquer spring tournament instead receive invitation longstanding rival nekoma high karasuno agree large train camp alongside notable volleyball team tokyo national level players play toughest team japan hope sharpen skills come new attack strengthen hinata kageyama attempt devise powerful weapon possibly break sturdiest block face chance victory senior players graduate members karasunos volleybal...
1        music accompany path human metronome prodigious pianist kousei arima pass mother saki arima kousei fall downward spiral render unable hear sound piano years later kousei avoid piano leave admirers rival live colorless life alongside friends tsubaki sawabe ryouta watari change meet beautiful violinist kaori miyazono stir world set journey face music base manga series shigatsu wa kimi uso approach story kouseis recovery discover music play note perfectly single melody bring fresh

In [16]:
# !pip install pyspellchecker
# spell = SpellChecker()
# for word in syn:
#   print(spell.unknown(word.split()))
#####################

#   from spellchecker import SpellChecker

# spell = SpellChecker()

# def correct_spellings(text):
#     corrected_text = []
#     misspelled_words = spell.unknown(text.split())
#     for word in text.split():
#         if word in misspelled_words:
#             corrected_text.append(spell.correction(word))
#         else:
#             corrected_text.append(word)
#     return " ".join(corrected_text)
        
# correct_spellings(syn)
# syn

In [17]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

syn = list(sent_to_words(syn))

In [18]:
# Create Dictionary
id2word = corpora.Dictionary(syn)

# Create Corpus
texts = syn

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
id2word[76]

'accompany'

In [20]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('agree', 1),
  ('aim', 1),
  ('alongside', 1),
  ('aoba', 1),
  ('archrival', 1),
  ('attack', 1),
  ('attempt', 2),
  ('block', 1),
  ('break', 1),
  ('camp', 1),
  ('chance', 1),
  ('come', 1),
  ('conquer', 1),
  ('devise', 1),
  ('differences', 1),
  ('efforts', 1),
  ('face', 1),
  ('follow', 1),
  ('formidable', 1),
  ('graduate', 1),
  ('harder', 1),
  ('high', 2),
  ('hinata', 1),
  ('hope', 2),
  ('instead', 1),
  ('interhigh', 1),
  ('invitation', 1),
  ('japan', 1),
  ('jousai', 1),
  ('kageyama', 1),
  ('karasuno', 2),
  ('karasunos', 1),
  ('large', 1),
  ('learn', 1),
  ('level', 1),
  ('longstanding', 1),
  ('mal', 1),
  ('members', 1),
  ('national', 1),
  ('nekoma', 1),
  ('new', 1),
  ('newincluding', 1),
  ('notable', 1),
  ('oikawa', 1),
  ('old', 1),
  ('opponents', 1),
  ('overcome', 1),
  ('participation', 1),
  ('play', 1),
  ('players', 2),
  ('possibly', 1),
  ('powerful', 1),
  ('receive', 1),
  ('refocus', 1),
  ('rewrite', 1),
  ('rival', 1),
  ('school'

In [21]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(n

In [22]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.041*"anime" + 0.028*"red" + 0.021*"moon" + 0.017*"stand" + 0.016*"bear" + '
  '0.014*"project" + 0.012*"rumor" + 0.011*"fire" + 0.011*"shadow" + '
  '0.010*"continent"'),
 (1,
  '0.026*"write" + 0.026*"mal" + 0.025*"find" + 0.025*"rewrite" + 0.021*"life" '
  '+ 0.020*"live" + 0.018*"girl" + 0.017*"time" + 0.015*"day" + 0.014*"begin"'),
 (2,
  '0.099*"source" + 0.043*"ann" + 0.033*"friends" + 0.031*"story" + '
  '0.027*"name" + 0.024*"new" + 0.021*"set" + 0.020*"boy" + 0.019*"place" + '
  '0.016*"years"'),
 (3,
  '0.038*"team" + 0.036*"japan" + 0.026*"train" + 0.022*"play" + 0.020*"tokyo" '
  '+ 0.016*"rival" + 0.014*"challenge" + 0.013*"manage" + 0.013*"air" + '
  '0.013*"instead"'),
 (4,
  '0.023*"wish" + 0.019*"night" + 0.018*"days" + 0.018*"pure" + 0.016*"magic" '
  '+ 0.012*"witch" + 0.011*"inuyasha" + 0.010*"queen" + 0.009*"party" + '
  '0.009*"cant"'),
 (5,
  '0.031*"king" + 0.020*"hero" + 0.017*"worlds" + 0.015*"float" + '
  '0.013*"military" + 0.013*"recap" + 0.012*"k

In [23]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=syn, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(n


Perplexity:  -9.821082461648253

Coherence Score:  0.4401434466914309


In [24]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.331108  0.220690       1        1  19.357514
2      0.206949  0.150208       2        1   9.175748
10     0.189253 -0.261017       3        1   9.159142
9      0.102577 -0.173061       4        1   6.241700
11     0.068867 -0.001463       5        1   6.112488
7      0.088745 -0.185147       6        1   4.718148
4     -0.037964 -0.029045       7        1   4.019063
3     -0.038161 -0.008412       8        1   3.980721
16    -0.058837  0.054402       9        1   3.950576
8     -0.058089  0.007273      10        1   3.763829
0     -0.076637  0.026002      11        1   3.384327
17    -0.070370  0.014294      12        1   3.378285
6     -0.074024  0.034986      13        1   3.195701
14    -0.074448  0.046141      14        1   3.110084
15    -0.065349  0.002099      15        1   3.076910
12    -0.092364  0.028448      16        1   2.947677
19    -0.083957  0.031563      17        1   2.875868
5     -0.076371 -0.004151      18        1   2.685403
18    -0.085142  0.025300      19        1   2.535009
13    -0.095787  0.020890      20        1   2.331808, topic_info=           Term         Freq        Total Category  logprob  loglift
993      source  6039.000000  6039.000000  Default  30.0000  30.0000
56       school  3527.000000  3527.000000  Default  29.0000  29.0000
137       world  3226.000000  3226.000000  Default  28.0000  28.0000
930         ann  2609.000000  2609.000000  Default  27.0000  27.0000
123      series  1761.000000  1761.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
2373         ii    68.727270    69.440010  Topic20  -5.4215   3.7482
633   political    67.965338    68.678078  Topic20  -5.4327   3.7481
1238   imperial    66.778990    67.491730  Topic20  -5.4503   3.7479
490       greet    62.714406    63.427147  Topic20  -5.5131   3.7472
3140      ruler    67.046981   172.228406  Topic20  -5.4463   2.8151

[695 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1414      4  0.998119  ability
517      17  0.999203  academy
565      10  0.225340      ace
565      16  0.767564      ace
2114      5  0.999006  achieve
...     ...       ...      ...
1169     15  0.996798      yui
5678      5  0.993099     yuki
6644     15  0.997277    yuuki
2772     10  0.996305     zeon
564      17  0.992072     zero

[749 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 11, 10, 12, 8, 5, 4, 17, 9, 1, 18, 7, 15, 16, 13, 20, 6, 19, 14])